<a href="https://colab.research.google.com/github/davis689/binder/blob/master/CHEM452/Transport_Properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import sympy as sp
import matplotlib.pyplot as plt
nd,V,A,v_avg,theta,m,T,f_v,mfp=sp.symbols('N_V,V,A,v_avg, theta,m,T,f_v,lambda',nonnegative=True,real=True)
kb=sp.symbols('k_B',real=True,positive=True,constant=True)
dt=sp.Symbol(r'\Delta t')
theta,phi,v=sp.symbols('theta,phi,v',nonnegative=True,real=True)
from scipy.constants import Boltzmann, gas_constant, Avogadro
k_b=Boltzmann
NA=Avogadro

In [ ]:
volume=sp.Eq(V,A*v*dt*sp.cos(theta))
N_cyl=sp.Eq(V*nd,nd*volume.rhs)
N_cyl #number in cylinder

We need to determine which molecules are moving with sufficient speed **and** direction to pass throught the area, A. Here we can use the Maxwell-Boltzmann distribution in spherical polar coordinates but, here, since we're going to want to limit the available trajectories of the molecules, we won't integrate over all possible angles. This leaves us with a $v^2sin\theta d\theta d\phi dv$ replacing the $4\pi v^2$ in our three-dimensional Maxwell-Boltzmann distribution function.

In [ ]:
mb3d=sp.Eq(f_v,sp.sqrt(m/(2*sp.pi*kb*T))**3*sp.exp(-m*v**2/(2*kb*T))*v**2*sp.sin(theta))
mb3d

Now we should be able to integrate this over all $\phi$ but only allow $\theta$ to range from 0 to $\frac{\pi}{2}$. Let's see what this gives...

In [ ]:
frac1=sp.integrate(sp.integrate(sp.integrate(mb3d.rhs,(phi,0,2*sp.pi)),(theta,0,sp.pi/2)),(v,0,sp.oo))
frac1.subs({kb:k_b,m:0.040,T:300}).evalf()

Is this result reasonable? 

If we multiply this version of the Maxwell-Boltzmann distribution by the number of molecules in our cylinder we get an expression that will give us the number of molecules moving with the speed and the direction that we choose.

In [ ]:
Nds=sp.symbols('N_{ds}',nonnegative=True,real=True)
expr1=sp.Eq(Nds,N_cyl.rhs*mb3d.rhs) #N_ds means N in the (d)irection and with the (s)peed to pass through the area, A.
expr1

Now let's simplify a little by adjusting to calculate the number that pass through area A per unit time. This quantity is called *flux* and we give it the symbol $J_z$ for flux in the $z$-direction.

In [ ]:
Jz=sp.symbols('J_z',real=True)
dflux=sp.Eq(Jz,expr1.rhs/A/dt)
dflux

This is effectively a kind of flux distribution function giving the flux in any direction (although it's not normalized and so doesn't give fraction of flux in any direction or something similar).

Now let's integrate over all molecules moving *up* with any speed and from any $\phi$ direction.

In [ ]:
flux=sp.integrate(sp.integrate(sp.integrate(dflux.rhs,(phi,0,2*sp.pi)),(theta,0,sp.pi/2)),(v,0,sp.oo),conds='none')# if we leave off conds='none', we still get the answer but also a second answer.
                                                                                                                  #probably because we didn't completely specify assumptions about some symbol (positive,real,etc)
                                                                                                                  #and there's probably a scenario in which the integral can't be evaluated. But that's not true for our case.
flux

If we factor out $\sqrt{\dfrac{8k_BT}{\pi m}}$ which is average speed we get $$J_z=\frac{1}{4}N_V\left<v\right>$$ where $N_V$ is the density of molecules.

We can get sympy to factor out at least some of the average speed (for some reason including the 8 doesn't work. But multiply the result by 2/2 and move the $2\sqrt2$ into the square root and we're left with the result above.

In [ ]:
print(sp.collect(flux,sp.sqrt(kb*T/sp.pi/m),evaluate=False)[sp.sqrt(kb*T/sp.pi/m)])

In [123]:
exprJz=nd*v_avg/4


Now this is not particularly informative. I doubt that many have a serious interest in calculating flux in terms of speed density of molecules but it is a step on the way to something more informative.

We will now pull another piece of information useful to understanding transport properties from out understanding of averages and mean free path.

The mean free path was determined to be the average distance a molecule travels between collisions. For transport properties, we are interested in average travel in a particular direction so knowing the mean free path in a particular direction, say along the $z$-axis would be useful. If $\theta$ is the angle from the $z$-axis, the component of vector in the direction of motion with length, $\lambda$, will be $\lambda cos\theta$.

We can use our flux 'distribution' function equation to average this component along the $z$-axis. There is no $v$ or $\phi$ dependence on this quantity. Since the 'distribution' function in not normalized we will need to normalize in order to use this expression to find the average. This can be accomplished by simply dividing by the integral of the distribution function. (If that is not clear, think of how we normalized the Maxwell-Boltzmann distribution. We multiplied by a constant and integrated over the entire range of the variables and set equal to 1. Solving for the constant of normalization means dividing 1 by the integral. So the normalization constant is really just the reciprocal of the integral.) Since the $v$ and $\phi$ integrals in the denominator and numerator will be the same, they cancel out leaving only the $\theta$ integrals. We're going to continue integrating only over $\theta$ of 0 to $\pi$/2 since we're looking for motion in one direction.

In [ ]:
sp.integrate(mfp*sp.cos(theta)*dflux.rhs,(theta,0,sp.pi/2))/sp.integrate(dflux.rhs,(theta,0,sp.pi/2))

So on average, a molecule travels 2/3$\lambda$ along the $z$-axis between collisions. This, of course, can happen in either direction along the $z$-axis even though we only calculated it in the positive direction.



So now we'll calculate the flux of molecules or of some property $q$ along the $z$ axis if the concentration is not uniform. We'll use the symbol, $N_q$ to stand for the number density of property $q$. $q$ can be just the molecules themselves (molecular diffusion), the momentum of molecules (viscosity), the movement of the thermal energy of molecules (thermal conductivity), etc. Transport of all of these can be handled in the same way.

 Imagine that the concentration of molecules (and whatever properties they carry) varies in some way that can be described by a Taylor series. For a concentration 2/3$\lambda$ away from the origin.

 $$N_q(z_0\pm\frac{2}{3}\lambda)=N_q(z_0)\pm\frac{2}{3}\lambda\left(\dfrac{dN_q}{dz}\right)_{z_0}+...$$

 Given this concentration profile, what will be the flux at $z_0$ given that some net number of molecules will enter the $z_0$ slice moving in the negative direction and some will enter from the negative side (moving in the positive direction). The difference between these quantities at $z_0+2/3\lambda$ and $z_0-2/3\lambda$ will give the flux at $z_0$.

In [ ]:
z=sp.symbols('z')
Nq=sp.Function('N_q')(z) # Define Nq as a function of z

Jzplus=exprJz.subs(nd,Nq-2*mfp/3*sp.diff(Nq,z)) # moving in the positive direction 
Jzminus=exprJz.subs(nd,Nq+2*mfp/3*sp.diff(Nq,z)) # moving in the negative direction
Jz=Jzplus-Jzminus
Jz.simplify()

For a given molecule at a given temperature, the net flux is equal to a constant multiplied by the change in the concentration with position or $$J_z=-D\left(\dfrac{dN_q}{dz}\right)$$ where D is the diffusion coefficient and the derivative is the concentration gradient. Flux is zero when there is no difference in concentration. Given a non-zero concentration gradient, flux occurs in the direction opposite the gradient. This relation is Fick's First [Law of Diffusion](https://en.wikipedia.org/wiki/Fick%27s_laws_of_diffusion).

As mentioned earlier, this works for diffusion of molecules or for properties carried by those molecules. In the base case, $N_q$ is just the number density of molecules. But we could modify it to be the density of energy carried by the molecules and we would be considering thermal conductivity instead. 

Even though, Fick's first law is reasonable and clear once you understand, it's doubtful that you've ever had reason to ask what the flux was for a system in terms of the concentration gradient. What might be closer to relevance to more people would be if we could determine the concentration of a system at a point as a function of time.

Consider a slab of a gas $dz$ thick and area $A$ on a side. Molecules are moving opposite a concentration gradient in the $z$ direction as indicated by Fick's first law. 

We set up and find the flux at $z_0$


In [ ]:
Jz,D,dz,t=sp.symbols('J_z,D,dz,t')
DJz=sp.Symbol(r'd J_z')
Jzdz=sp.Symbol(r'J_{z+ dz}')

Fick1=sp.Eq(Jz,-D*sp.diff(Nq,z))
Fick1

And we can also find the flux at a short disance away from $z_0$. We'll again use a Taylor series to approximate our concentration gradient, this time just using $dz$ for the change in $z$ rather than $2/3\lambda$. So we have $$N_q(z_0+ dz)=N_q(z_0)+ dz\left(\dfrac{dN_q}{dz}\right)_{z_0}+...$$ Substitute into $J_{z+dz}$ and get

In [ ]:
Fick1dz=sp.Eq(Jzdz,Fick1.rhs.subs(Nq,Nq+dz*sp.diff(Nq,z)))
Fick1dz

Now we have flux at $z_0$ and $z_0 + dz$. If we take the difference, we'll have the change in flux or the difference between the molecules entering our slab and those leaving it. This difference will be related to the net change in the number of molecules in our slab. Flux is molecules/area/time. It might be conceptually easier to multiply the flux by area to get molecules/time. That way the difference between $AJ_z$ and $AJ_{z+dz}$ is the change of molecules in our slab per second.

In [ ]:
Fick2a=sp.Eq(A*DJz,A*(Fick1.rhs-Fick1dz.rhs))
Fick2a

Combining and simplifying terms we get

In [ ]:
Fick2a.simplify()

So if $A\,dJ_z$ is the number of molecules entering the slab per second and, if $Adz$ is the volume of the slab, dividing $A\,dJ_z$ by $Adz$ will give molecules per volume per second or $\frac{dN_q}{dt}$

In [ ]:
Nq=sp.Function('N_q')(z,t)
Fick2b=sp.Eq(sp.diff(Nq,t),Fick2a.rhs/(dz*A))
Fick2b.simplify()

This is Fick's second law in one spatial dimension. It depends on both time and position. We can worry about solving it later. A solution for the case of an initial concentration $N_{z_0}$ in molecules/area all at $z_0$ at the initial time with diffusion ocurring in only the positive $z$ direction is

In [ ]:
Nz0=sp.symbols('N_{z_0}')
Fick2=sp.Eq(Nq,Nz0/(2*sp.sqrt(sp.pi*D*t))*sp.exp(-z**2/(4*D*t)))
Fick2

In [ ]:
#from numpy import linspace
#fick2nd=sp.lambdify([z,t,D],Fick2.rhs.subs(Nz0,.1).evalf())
#zz=linspace(0,1,1000)


In [ ]:
from sympy.plotting import plot
plt1=plot(Fick2.rhs.subs({t:20,D:1e-5,Nz0:.1,z:zz}),xlim=(0,1),ylim=(0,1),ylabel="$N_q$(z)",show=False,label='20 s',legend=True)
plt2=plot(Fick2.rhs.subs({t:100,D:1e-5,Nz0:.1,z:zz}),xlim=(0,1),ylim=(0,1),ylabel="$N_q$(z)",line_color='g',show=False,label='100 s')
plt3=plot(Fick2.rhs.subs({t:500,D:1e-5,Nz0:.1,z:zz}),xlim=(0,1),ylim=(0,1),ylabel="$N_q$(z)",line_color='orange',show=False,label='500 s')
plt4=plot(Fick2.rhs.subs({t:2500,D:1e-5,Nz0:.1,z:zz}),xlim=(0,1),ylim=(0,1),ylabel="$N_q$(z)",line_color='r',show=False,label='2500 s')
plt1.extend(plt2)
plt1.extend(plt3)
plt1.extend(plt4)
plt1.show()